In [139]:
import random
import requests
from bs4 import BeautifulSoup
import time
import csv
from fake_useragent import UserAgent

In [142]:
with open('proxies.csv', 'r') as f:
    csv_reader = csv.reader(f)
    ips = list(csv_reader)

ip_addresses = [ip[0] for ip in ips]

session = requests.Session()

def proxy_request(url, ip_addresses):
    while True:
        try:
            ua = UserAgent()    
            headers = {"Accept-Language": "en-US,en;q=0.5", 'User-Agent': ua.random}
            proxy = random.randint(0, len(ip_addresses) - 1)
            proxies = {"http": ip_addresses[proxy]}
            response = session.get(url, proxies=proxies, timeout=5, headers=headers)
            if response.status_code == 200:
                break
            elif response.status_code == 429:
                print(f"Rate limit exceeded. Waiting before retrying...")
                time.sleep(10)
            else:
                print(f"Request failed with status code: {response.status_code}")
                time.sleep(random.uniform(1, 3))
        except Exception as e:
            print(f"Error: {e}")

    return response

In [143]:
url = 'https://www.gsmarena.com/makers.php3'
response = proxy_request(url, ip_addresses)
content = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

Rate limit exceeded. Waiting before retrying...
Rate limit exceeded. Waiting before retrying...


KeyboardInterrupt: 

In [ ]:
prefix = 'https://www.gsmarena.com/makers.php3/'
maker_urls = dict()
query = list(map(lambda x: x.find('a'), content.find('table').find_all('td')))
maker_urls = dict(zip(list(map(lambda x: x.text.split()[0], query)), list(map(lambda x: prefix + x.attrs['href'], query))))

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
maker_urls

{'Acer100': 'https://www.gsmarena.com/makers.php3/acer-phones-59.php',
 'alcatel409': 'https://www.gsmarena.com/makers.php3/alcatel-phones-5.php',
 'Allview157': 'https://www.gsmarena.com/makers.php3/allview-phones-88.php',
 'Amazon25': 'https://www.gsmarena.com/makers.php3/amazon-phones-76.php',
 'Amoi47': 'https://www.gsmarena.com/makers.php3/amoi-phones-28.php',
 'Apple118': 'https://www.gsmarena.com/makers.php3/apple-phones-48.php',
 'Archos43': 'https://www.gsmarena.com/makers.php3/archos-phones-90.php',
 'Asus203': 'https://www.gsmarena.com/makers.php3/asus-phones-46.php',
 'AT&T4': 'https://www.gsmarena.com/makers.php3/at&t-phones-57.php',
 'Benefon9': 'https://www.gsmarena.com/makers.php3/benefon-phones-15.php',
 'BenQ35': 'https://www.gsmarena.com/makers.php3/benq-phones-31.php',
 'BenQ-Siemens28': 'https://www.gsmarena.com/makers.php3/benq_siemens-phones-42.php',
 'Bird61': 'https://www.gsmarena.com/makers.php3/bird-phones-34.php',
 'BlackBerry92': 'https://www.gsmarena.com/m

In [ ]:
prefix = 'https://www.gsmarena.com/'
for i, maker in enumerate(maker_urls):
    response = proxy_request(maker_urls[maker], ip_addresses)
    content = BeautifulSoup(response.content, 'html.parser')
    query = content.find('div', {'class': 'makers'}).find_all('a')
    maker_urls[maker] = dict(map(lambda x: (x.text, prefix + x.attrs['href']), query))
    print(maker_urls[maker])

{'Chromebook Tab 10': 'https://www.gsmarena.com/acer_chromebook_tab_10-9139.php', 'Iconia Talk S': 'https://www.gsmarena.com/acer_iconia_talk_s-8306.php', 'Liquid Z6 Plus': 'https://www.gsmarena.com/acer_liquid_z6_plus-8305.php', 'Liquid Z6': 'https://www.gsmarena.com/acer_liquid_z6-8304.php', 'Iconia Tab 10 A3-A40': 'https://www.gsmarena.com/acer_iconia_tab_10_a3_a40-8080.php', 'Liquid X2': 'https://www.gsmarena.com/acer_liquid_x2-8034.php', 'Liquid Jade 2': 'https://www.gsmarena.com/acer_liquid_jade_2-7956.php', 'Liquid Zest Plus': 'https://www.gsmarena.com/acer_liquid_zest_plus-8059.php', 'Liquid Zest': 'https://www.gsmarena.com/acer_liquid_zest-7955.php', 'Predator 8': 'https://www.gsmarena.com/acer_predator_8-7750.php', 'Liquid Jade Primo': 'https://www.gsmarena.com/acer_liquid_jade_primo-7650.php', 'Liquid Z330': 'https://www.gsmarena.com/acer_liquid_z330-7530.php', 'Liquid Z320': 'https://www.gsmarena.com/acer_liquid_z320-7531.php', 'Liquid Z630S': 'https://www.gsmarena.com/acer

In [ ]:
for brand in maker_urls:    
    for url in maker_urls[brand].values():
        url = url.replace('https://www.gsmarena.com/makers.php3', 'https://www.gsmarena.com/makers.php3/')
        print(url)
        response = proxy_request(url, ip_addresses)
        soup = BeautifulSoup(response.content, 'html.parser')
        table_list = soup.find_all('table')

        # Initialize a dictionary to hold the header and its associated data
        info_dict_with_headers = {}

        for table in table_list:
            # Attempt to find the table header
            header = table.find('th')
            if header:
                header_text = header.get_text(strip=True)
            else:
                # If no <th> is found, or there is a need to identify the header differently, adjust here
                header_text = "Unknown"  # Placeholder if no header is found or defined differently

            # Initialize the dictionary for this header if not already present
            if header_text not in info_dict_with_headers:
                info_dict_with_headers[header_text] = []

            last_key = None  # Keep track of the last non-empty key for rows without a 'ttl'

            # Iterate through each row in the table
            for row in table.find_all('tr'):
                ttl_element = row.find("td", class_="ttl")
                nfo_element = row.find("td", class_="nfo")

                if ttl_element and ttl_element.get_text(strip=True):
                    # If ttl_element has text, update last_key
                    key = ttl_element.get_text(strip=True)
                    last_key = key
                else:
                    # If ttl_element is empty, use the last_key
                    key = last_key

                if nfo_element:
                    value = nfo_element.get_text(strip=True)
                    # Append the key-value pair under the current header
                    if key:
                        info_dict_with_headers[header_text].append({key: value})
                    else:
                        # Handle case where there's a value without a key (could append to last key's value or handle as needed)
                        pass
        print(info_dict_with_headers)
                        

https://www.gsmarena.com/acer_chromebook_tab_10-9139.php
{'Network': [{'Technology': 'No cellular connectivity'}, {'2G bands': 'N/A'}, {'GPRS': 'No'}, {'EDGE': 'No'}], 'Launch': [{'Announced': '2018, March'}, {'Status': 'Available. Released 2018, July'}], 'Body': [{'Dimensions': '238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in)'}, {'Weight': '544.3 g (1.20 lb)'}, {'SIM': 'No'}, {'SIM': 'Stylus'}], 'Display': [{'Type': 'IPS LCD'}, {'Size': '9.7 inches, 291.4 cm2(~71.0% screen-to-body ratio)'}, {'Resolution': '1536 x 2048 pixels, 4:3 ratio (~264 ppi density)'}], 'Platform': [{'OS': 'Chrome OS'}, {'Chipset': 'Rockchip RK3399'}, {'CPU': 'Hexa-core (4x Cortex-A53 & 2x Cortex-A72)'}], 'Memory': [{'Card slot': 'microSDXC (dedicated slot)'}, {'Internal': '32GB 4GB RAM'}], 'Main Camera': [{'Single': '5 MP'}, {'Video': '720p'}], 'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}], 'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'}, {'3.5mm jack': 'Yes'}], 'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/

KeyboardInterrupt: 

In [ ]:
info_dict_with_headers

{}